In [319]:
import pickle
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from random import shuffle

In [321]:
with open("data.pickle", "rb") as f:
    dataset = pickle.load(f)

x_test, y_test = dataset["test"][:, 0], dataset["test"][:, 1]

shuffle(dataset["train"])
x_train_big = dataset["train"][:, 0]
y_train_big = dataset["train"][:, 1]
x_trains = np.array([x_train_big[i : i + x_train_big.size // 15] for i in range(15)])
y_trains = np.array([y_train_big[i : i + y_train_big.size // 15] for i in range(15)])

In [353]:
for degree in range(1, 11):
    polynomial = PolynomialFeatures(degree)
    res = []

    for x_train, y_train in zip(x_trains, y_trains):
        model = LinearRegression()
        model.fit(polynomial.fit_transform(x_train.reshape(-1, 1)), y_train)
        res.append(model.predict(polynomial.fit_transform(x_test.reshape(-1, 1))))

    variance = np.var(res, axis=0)
    square = np.vectorize(lambda x: x**2)
    bias = y_test - np.mean(res, axis=0)
    bias_square = square(bias)
    mse = np.mean(square([y_test - y_pred for y_pred in res]), axis=0)

    average = {
        "bias": np.mean(bias),
        "bias_square": np.mean(bias_square),
        "variance": np.mean(variance),
        "mse": np.mean(mse),
    }
    
    print(average["variance"] + average["bias_square"], average["mse"])

1.164630571268875 1.1646305712688751
1.0800678556447831 1.080067855644783
0.2496278089928526 0.2496278089928527
0.40850040621019856 0.40850040621019834
0.4497713891123903 0.4497713891123908
0.5033103970313434 0.5033103970313433
0.5545730007143946 0.5545730007143961
0.4655449092011339 0.465544909201134
0.35793042507822476 0.3579304250782255
0.49213835872229306 0.4921383587222934
